# Basic LSDB/HiPSCat Functionality

In this notebook we will demonstrate analysis of single hipscats
* querying based off of columnar information
* querying based off of conesearches
* utilize the `dask.dataframe` parallelized pandas API to concatenate methods

In [1]:
import lsdb
from pathlib import Path
path_to_local_catalog = f"{Path.home()}/shared/lincc-frameworks/data/ADASS_Tutorial/gaia_exA"

gaia = lsdb.read_hipscat(path_to_local_catalog)
gaia

,solution_id,designation,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,pm,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_bad_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,astrometric_primary_flag,nu_eff_used_in_astrometry,pseudocolour,pseudocolour_error,ra_pseudocolour_corr,dec_pseudocolour_corr,parallax_pseudocolour_corr,pmra_pseudocolour_corr,pmdec_pseudocolour_corr,astrometric_matched_transits,visibility_periods_used,astrometric_sigma5d_max,matched_transits,new_matched_transits,matched_transits_removed,ipd_gof_harmonic_amplitude,ipd_gof_harmonic_phase,ipd_frac_multi_peak,ipd_frac_odd_win,ruwe,scan_direction_strength_k1,scan_direction_strength_k2,scan_direction_strength_k3,scan_direction_strength_k4,scan_direction_mean_k1,scan_direction_mean_k2,scan_direction_mean_k3,scan_direction_mean_k4,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,phot_bp_n_contaminated_transits,phot_bp_n_blended_transits,phot_rp_n_contaminated_transits,phot_rp_n_blended_transits,phot_proc_mode,bp_rp,bp_g,g_rp,radial_velocity,radial_velocity_error,rv_method_used,rv_nb_transits,rv_nb_deblended_transits,rv_visibility_periods_used,rv_expected_sig_to_noise,rv_renormalised_gof,rv_chisq_pvalue,rv_time_duration,rv_amplitude_robust,rv_template_teff,rv_template_logg,rv_template_fe_h,rv_atm_param_origin,vbroad,vbroad_error,vbroad_nb_transits,grvs_mag,grvs_mag_error,grvs_mag_nb_transits,rvs_spec_sig_to_noise,phot_variable_flag,l,b,ecl_lon,ecl_lat,in_qso_candidates,in_galaxy_candidates,non_single_star,has_xp_continuous,has_xp_sampled,has_rvs,has_epoch_photometry,has_epoch_rv,has_mcmc_gspphot,has_mcmc_msc,in_andromeda_survey,classprob_dsc_combmod_quasar,classprob_dsc_combmod_galaxy,classprob_dsc_combmod_star,teff_gspphot,teff_gspphot_lower,teff_gspphot_upper,logg_gspphot,logg_gspphot_lower,logg_gspphot_upper,mh_gspphot,mh_gspphot_lower,mh_gspphot_upper,distance_gspphot,distance_gspphot_lower,distance_gspphot_upper,azero_gspphot,azero_gspphot_lower,azero_gspphot_upper,ag_gspphot,ag_gspphot_lower,ag_gspphot_upper,ebpminrp_gspphot,ebpminrp_gspphot_lower,ebpminrp_gspphot_upper,libname_gspphot,Norder,Dir,Npix
npartitions=23,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,string,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,float64,float64,float64,float64,int64,bool,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,int64,int64,int64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,int64,float64,float64,float64,float64,int64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,string,float64,float64,float64,float64,bool,bool,int64,bool,bool,bool,bool,bool,bool,bool,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float6

In [ ]:
dict(gaia.dtypes)

In [ ]:
#load the catalog as a dask dataframe

gaia = lsdb.read_hipscat(path_to_local_catalog, columns=['ra', 'dec', 'pmra', 'pmdec'])
gaia

### Loading a hipscat from a pandas DataFrame

Say you have a small enough dataset you can load into a single dataframe. You can instantiate a hipscat catalog from just a single dataframe!

In [ ]:
import pandas as pd

path_to_parquet = f"{Path.home()}/shared/lincc-frameworks/data/ADASS_Tutorial/GaiaSource_000000-003111.parquet"
df = pd.read_parquet(path_to_parquet, columns=['ra', 'dec', 'pmra', 'pmdec'])

df_hipscat = lsdb.from_dataframe(df, catalog_name="GaiaSource_1", ra_column="ra", dec_column="dec")
df_hipscat

### Saving a `hipscat.Catalog` result to disk


In [ ]:
df_hipscat.to_hipscat(path=f"{Path.home()}/shared/lincc-frameworks/data/ADASS_Tutorial/output")

## Lazy Evaluation and Parallelized python infrastructure
Our computations rely on the `dask.dataframe` API, which as you can see hasn't evaluated anything. For all `dask.dataframe` computations, the scheduled process aren't executed until you perform a `.compute()` at the end of the line of code. 

When utilizing this API, we can define a client to specify the number of workers, the address/port in two ways:
* with `dask.distributed` [client](https://distributed.dask.org/en/latest/client.html#dask)
* with `dask_on_ray`'s [scheduler](https://docs.ray.io/en/latest/ray-more-libs/dask-on-ray.html#scheduler)

We have personally found Dask on Ray to be better at scheduling and memory management, so we encourage others to use that.

In [ ]:
from dask.distributed import Client
import ray
from ray.util.dask import enable_dask_on_ray, disable_dask_on_ray

class lsdb_client():
    def __init__(self, dask_on_ray=True, num_workers=4):
        self.dask_on_ray = dask_on_ray
        self.num_workers=num_workers
        self.start()

    def start(self):

        if self.dask_on_ray:
            self.client = ray.init(
                num_cpus=self.num_workers
            )
            enable_dask_on_ray()
        else:
            self.client = Client(n_workers=self.num_workers, threads_per_worker=1)

    def shutdown(self):
        if self.dask_on_ray:
            disable_dask_on_ray()
            ray.shutdown()
        else:
            self.client.close()

client = lsdb_client(dask_on_ray=True)

## Actually performing some computations

Now that we have our parallelized framework set up. Let's analyze our dataset with the LSDB codebase:
* performing cone searches
* querying based off of column info
* assigning columns
    * concatenating all this functionality

In [5]:
#cone search
cone_search = gaia.cone_search(
    ra=45,
    dec=25,
    radius=1
)
cone_search

,solution_id,designation,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,pm,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_bad_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,astrometric_primary_flag,nu_eff_used_in_astrometry,pseudocolour,pseudocolour_error,ra_pseudocolour_corr,dec_pseudocolour_corr,parallax_pseudocolour_corr,pmra_pseudocolour_corr,pmdec_pseudocolour_corr,astrometric_matched_transits,visibility_periods_used,astrometric_sigma5d_max,matched_transits,new_matched_transits,matched_transits_removed,ipd_gof_harmonic_amplitude,ipd_gof_harmonic_phase,ipd_frac_multi_peak,ipd_frac_odd_win,ruwe,scan_direction_strength_k1,scan_direction_strength_k2,scan_direction_strength_k3,scan_direction_strength_k4,scan_direction_mean_k1,scan_direction_mean_k2,scan_direction_mean_k3,scan_direction_mean_k4,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,phot_bp_n_contaminated_transits,phot_bp_n_blended_transits,phot_rp_n_contaminated_transits,phot_rp_n_blended_transits,phot_proc_mode,bp_rp,bp_g,g_rp,radial_velocity,radial_velocity_error,rv_method_used,rv_nb_transits,rv_nb_deblended_transits,rv_visibility_periods_used,rv_expected_sig_to_noise,rv_renormalised_gof,rv_chisq_pvalue,rv_time_duration,rv_amplitude_robust,rv_template_teff,rv_template_logg,rv_template_fe_h,rv_atm_param_origin,vbroad,vbroad_error,vbroad_nb_transits,grvs_mag,grvs_mag_error,grvs_mag_nb_transits,rvs_spec_sig_to_noise,phot_variable_flag,l,b,ecl_lon,ecl_lat,in_qso_candidates,in_galaxy_candidates,non_single_star,has_xp_continuous,has_xp_sampled,has_rvs,has_epoch_photometry,has_epoch_rv,has_mcmc_gspphot,has_mcmc_msc,in_andromeda_survey,classprob_dsc_combmod_quasar,classprob_dsc_combmod_galaxy,classprob_dsc_combmod_star,teff_gspphot,teff_gspphot_lower,teff_gspphot_upper,logg_gspphot,logg_gspphot_lower,logg_gspphot_upper,mh_gspphot,mh_gspphot_lower,mh_gspphot_upper,distance_gspphot,distance_gspphot_lower,distance_gspphot_upper,azero_gspphot,azero_gspphot_lower,azero_gspphot_upper,ag_gspphot,ag_gspphot_lower,ag_gspphot_upper,ebpminrp_gspphot,ebpminrp_gspphot_lower,ebpminrp_gspphot_upper,libname_gspphot,Norder,Dir,Npix
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,string,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,float64,float64,float64,float64,int64,bool,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,int64,int64,int64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,int64,float64,float64,float64,float64,int64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,string,float64,float64,float64,float64,bool,bool,int64,bool,bool,bool,bool,bool,bool,bool,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64

In [6]:
cone_search.compute()

/astro/users/sdwyatt/git-clones/lsdb/src/lsdb/core/cone_search.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_frame["_CONE_SEP"] = df_separations


,solution_id,designation,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,...,ag_gspphot,ag_gspphot_lower,ag_gspphot_upper,ebpminrp_gspphot,ebpminrp_gspphot_lower,ebpminrp_gspphot_upper,libname_gspphot,Norder,Dir,Npix
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
220509169856282624,1636148068921376768,Gaia DR3 110254559267623552,110254559267623552,35809249,2016.0,45.247138,0.517697,24.034937,0.459174,0.010570,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3,0,12
220509188281860096,1636148068921376768,Gaia DR3 110254597922571392,110254597922571392,1364760889,2016.0,45.263956,1.151917,24.033004,1.002196,-0.189736,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3,0,12
220509226559078400,1636148068921376768,Gaia DR3 110254597922240000,110254597922240000,529128058,2016.0,45.260434,2.700799,24.039723,4.025234,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3,0,12
220509226651353088,1636148068921376768,Gaia DR3 110254597922915840,110254597922915840,1376481945,2016.0,45.260319,0.078819,24.039885,0.076077,4.208077,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3,0,12
220513018612547584,1636148068921376768,Gaia DR3 110256487708534272,110256487708534272,910496336,2016.0,45.098642,0.069134,24.011344,0.060984,0.233408,...,0.1289,0.0997,0.1788,0.0702,0.0542,0.0975,MARCS,3,0,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230528647943946240,1636148068921376768,Gaia DR3 115264312200703360,115264312200703360,52813885,2016.0,44.998439,0.169938,25.986789,0.180600,0.097190,...,0.0345,0.0064,0.1538,0.0189,0.0035,0.0842,MARCS,3,0,12
230528747667718144,1636148068921376768,Gaia DR3 115264350856751232,115264350856751232,1240476207,2016.0,44.965760,0.068382,25.989048,0.060044,0.955577,...,0.5531,0.5280,0.5740,0.2928,0.2798,0.3038,PHOENIX,3,0,12
230528771755606016,1636148068921376768,Gaia DR3 115264380920180352,115264380920180352,1395520085,2016.0,44.976662,0.902233,25.990917,0.864511,0.848256,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3,0,12


### Now lets perform the same computation but let's use the LSDB api to create a new column

by using the `dask.dataframe.assign` method

In [7]:
import numpy as np

gaia.cone_search(
    ra=45,
    dec=25,
    radius=1
).assign(
    pm=lambda x: np.sqrt(x['pmra']**2 + x['pmdec']**2)
).compute()

/astro/users/sdwyatt/git-clones/lsdb/src/lsdb/core/cone_search.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_frame["_CONE_SEP"] = df_separations


,solution_id,designation,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,...,ag_gspphot,ag_gspphot_lower,ag_gspphot_upper,ebpminrp_gspphot,ebpminrp_gspphot_lower,ebpminrp_gspphot_upper,libname_gspphot,Norder,Dir,Npix
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
220509169856282624,1636148068921376768,Gaia DR3 110254559267623552,110254559267623552,35809249,2016.0,45.247138,0.517697,24.034937,0.459174,0.010570,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3,0,12
220509188281860096,1636148068921376768,Gaia DR3 110254597922571392,110254597922571392,1364760889,2016.0,45.263956,1.151917,24.033004,1.002196,-0.189736,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3,0,12
220509226559078400,1636148068921376768,Gaia DR3 110254597922240000,110254597922240000,529128058,2016.0,45.260434,2.700799,24.039723,4.025234,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3,0,12
220509226651353088,1636148068921376768,Gaia DR3 110254597922915840,110254597922915840,1376481945,2016.0,45.260319,0.078819,24.039885,0.076077,4.208077,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3,0,12
220513018612547584,1636148068921376768,Gaia DR3 110256487708534272,110256487708534272,910496336,2016.0,45.098642,0.069134,24.011344,0.060984,0.233408,...,0.1289,0.0997,0.1788,0.0702,0.0542,0.0975,MARCS,3,0,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230528647943946240,1636148068921376768,Gaia DR3 115264312200703360,115264312200703360,52813885,2016.0,44.998439,0.169938,25.986789,0.180600,0.097190,...,0.0345,0.0064,0.1538,0.0189,0.0035,0.0842,MARCS,3,0,12
230528747667718144,1636148068921376768,Gaia DR3 115264350856751232,115264350856751232,1240476207,2016.0,44.965760,0.068382,25.989048,0.060044,0.955577,...,0.5531,0.5280,0.5740,0.2928,0.2798,0.3038,PHOENIX,3,0,12
230528771755606016,1636148068921376768,Gaia DR3 115264380920180352,115264380920180352,1395520085,2016.0,44.976662,0.902233,25.990917,0.864511,0.848256,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3,0,12


### Let's go a little bit further and cull our data based off of our calculation

by using the `dask.dataframe.query` method

In [8]:
gaia.cone_search(
    ra=45,
    dec=25,
    radius=1
).assign(
    pm=lambda x: np.sqrt(x['pmra']**2 + x['pmdec']**2)
).query(
    'pm > 20'
).compute()

/astro/users/sdwyatt/git-clones/lsdb/src/lsdb/core/cone_search.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_frame["_CONE_SEP"] = df_separations
/astro/users/sdwyatt/git-clones/lsdb/src/lsdb/core/cone_search.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_frame["_CONE_SEP"] = df_separations
/astro/users/sdwyatt/git-clones/lsdb/src/lsdb/core/cone_search.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Con

,solution_id,designation,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,...,ag_gspphot,ag_gspphot_lower,ag_gspphot_upper,ebpminrp_gspphot,ebpminrp_gspphot_lower,ebpminrp_gspphot_upper,libname_gspphot,Norder,Dir,Npix
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
61640737678163968,1636148068921376768,Gaia DR3 30820346013728768,30820346013728768,865357516,2016.0,45.734149,0.359984,15.038903,0.324586,1.609512,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3,0,3
61641903476572160,1636148068921376768,Gaia DR3 30820930129672704,30820930129672704,859551127,2016.0,45.659815,0.836707,15.047160,0.808083,2.655579,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3,0,3
61713353717841920,1636148068921376768,Gaia DR3 30856664257848064,30856664257848064,1156308566,2016.0,46.412348,0.028461,15.088996,0.025108,2.318999,...,0.4269,0.4143,0.4382,0.2274,0.2206,0.2334,PHOENIX,3,0,3
61746076134146048,1636148068921376768,Gaia DR3 30873019493342464,30873019493342464,1118890839,2016.0,46.945531,0.028463,15.181701,0.024255,2.150422,...,0.4267,0.4143,0.4390,0.2295,0.2228,0.2362,MARCS,3,0,3
61746321865834496,1636148068921376768,Gaia DR3 30873152636942080,30873152636942080,615579983,2016.0,46.978331,0.252900,15.200068,0.215942,0.296913,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280526417639243776,1636148068921376768,Gaia DR3 140263190823663104,140263190823663104,1171903569,2016.0,43.124811,0.056142,34.851164,0.048647,0.606271,...,0.0204,0.0064,0.0545,0.0111,0.0035,0.0296,MARCS,4,0,62
280526452036730880,1636148068921376768,Gaia DR3 140263220886859520,140263220886859520,1197540563,2016.0,43.141243,0.085212,34.857170,0.076260,1.523674,...,0.7788,0.7606,0.8037,0.4432,0.4334,0.4562,MARCS,4,0,62
280527499106975744,1636148068921376768,Gaia DR3 140263736282930816,140263736282930816,30021565,2016.0,43.046346,0.186350,34.850377,0.167840,0.013631,...,0.0053,0.0015,0.0137,0.0029,0.0008,0.0074,PHOENIX,4,0,62


### Other spatial querying methods: Polygon search

Here we give examples on querying a catalog based on results the lie within a polygon.

In [9]:
import astropy
import numpy as np

spherical_polygon = [
    [40, 50, 50, 40], #RA's
    [10, 10, 20, 20], #DECS
]
cartesian_polygon = astropy.coordinates.spherical_to_cartesian(
    1,                                #radial component
    np.deg2rad(spherical_polygon[1]), #LAT - DECs
    np.deg2rad(spherical_polygon[0]), #LON - RAs
)
gaia.polygon_search(
    np.array(cartesian_polygon).T
).compute()

/astro/users/sdwyatt/git-clones/lsdb/src/lsdb/core/polygon_search.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_frame["_POLYGON_PIX"] = hp.ang2pix(
/astro/users/sdwyatt/git-clones/lsdb/src/lsdb/core/polygon_search.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_frame["_POLYGON_PIX"] = hp.ang2pix(
/astro/users/sdwyatt/git-clones/lsdb/src/lsdb/core/polygon_search.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performa

,solution_id,designation,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,...,ag_gspphot,ag_gspphot_lower,ag_gspphot_upper,ebpminrp_gspphot,ebpminrp_gspphot_lower,ebpminrp_gspphot_upper,libname_gspphot,Norder,Dir,Npix
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
28689564746907648,1636148068921376768,Gaia DR3 14344778452281472,14344778452281472,277832891,2016.0,49.793463,0.400110,9.940486,0.384873,0.795056,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3,0,1
28689674696392704,1636148068921376768,Gaia DR3 14344812812064128,14344812812064128,1480189250,2016.0,49.796362,0.476262,9.956280,0.439261,1.162955,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3,0,1
28689723622948864,1636148068921376768,Gaia DR3 14344851466924160,14344851466924160,437749536,2016.0,49.782715,0.063849,9.954584,0.055371,1.747702,...,0.4107,0.3904,0.4341,0.2236,0.2126,0.2363,PHOENIX,3,0,1
28689745529798656,1636148068921376768,Gaia DR3 14344851466924544,14344851466924544,837791922,2016.0,49.772499,0.467811,9.957309,0.523977,-0.073752,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3,0,1
28689783672799232,1636148068921376768,Gaia DR3 14344885826005376,14344885826005376,39658137,2016.0,49.795413,0.398694,9.961531,0.360995,-0.010047,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216359997037084672,1636148068921376768,Gaia DR3 108179986984680960,108179986984680960,1007410,2016.0,44.373131,0.154154,20.095981,0.131536,0.537729,...,1.9950,1.9688,2.0281,1.1197,1.1066,1.1363,MARCS,3,0,12
216360113886199808,1636148068921376768,Gaia DR3 108180055704158080,108180055704158080,1637077557,2016.0,44.351074,0.389662,20.097965,0.326595,-0.017709,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3,0,12
216360180240089088,1636148068921376768,Gaia DR3 108180094359279872,108180094359279872,84297966,2016.0,44.362965,0.395488,20.106261,0.350410,0.821254,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3,0,12


### Pretty amazing results with what you can do with 4 cores and the 1.8Bn object Gaia catalog

Now lets shut down our client

In [10]:
client.shutdown()

NameError: name 'client' is not defined